In [116]:
import tensorflow as tf
import cv2
import numpy as np
import os
import pandas as pd
from tensorflow_hub import load


movenet = load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
def detect_keypoints(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image, (192, 192))
    image_resized = np.expand_dims(image_resized, axis=0).astype(np.int32)  # Convert to int32

    outputs = movenet.signatures['serving_default'](tf.constant(image_resized, dtype=tf.int32))
    keypoints = outputs['output_0'].numpy().reshape(-1, 3)  # 17 keypoints (x, y, confidence)
    return keypoints

data_folders = {"train": "./train", "test": "./test", "valid": "./valid"}
output_csv = "squat_keypoints.csv"

data = []
for dataset, path in data_folders.items():
    for label in ["correct", "incorrect"]:
        label_path = os.path.join(path, label)
        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            keypoints = detect_keypoints(img_path)
            row = [dataset, img_name]
            for i in range(17):
                row.extend([keypoints[i][0], keypoints[i][1], keypoints[i][2]])
            row.append(label)
            data.append(row)

columns = ["dataset", "image_name"] + [f"{coord}{i+1}" for i in range(17) for coord in ["x", "y", "c"]] + ["label"]
df = pd.DataFrame(data, columns=columns)
df.to_csv(output_csv, index=False)
print(f"Keypoints saved to {output_csv}")

Keypoints saved to squat_keypoints.csv


In [117]:
import pandas as pd
import torch
import torch.nn.functional as F
import torch.onnx
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool

data_path = "squat_keypoints.csv"
df = pd.read_csv(data_path)

edges = torch.tensor([
    [0, 1], [0, 2], [1, 3], [2, 4], [0, 5], [0, 6], [5, 7],
    [7, 9], [6, 8], [8, 10], [5, 6], [5, 11], [6, 12], [11, 13],
    [13, 15], [12, 14], [14, 16], [11, 12]
], dtype=torch.long).t()

graphs = []
for _, row in df.iterrows():
    keypoints = [[row[f"x{i+1}"], row[f"y{i+1}"], row[f"c{i+1}"]] for i in range(17)]
    x = torch.tensor(keypoints, dtype=torch.float)  # Node features
    y = torch.tensor([1 if row['label'] == 'correct' else 0], dtype=torch.long)  # Label
    graph = Data(x=x, edge_index=edges, y=y)
    graphs.append(graph)

print(f"Processed {len(graphs)} graphs for GNN training.")

train_size = int(0.8 * len(graphs))
train_graphs, test_graphs = graphs[:train_size], graphs[train_size:]

train_loader = DataLoader(train_graphs, batch_size=16, shuffle=True)
test_loader = DataLoader(test_graphs, batch_size=16, shuffle=False)

class GNNClassifier(torch.nn.Module):
    def __init__(self):
        super(GNNClassifier, self).__init__()
        self.conv1 = GCNConv(3, 16)  # Input features: (x, y, confidence)
        self.conv2 = GCNConv(16, 32)
        self.fc = torch.nn.Linear(32, 2)  # Binary classification

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)  # Pooling to get graph-level representation
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GNNClassifier().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

print("Training the model...")
for epoch in range(500):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch)  # Fix forward pass
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/500, Loss: {total_loss / len(train_loader):.4f}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index, batch.batch)  # Fix forward pass
        pred = out.argmax(dim=1)
        correct += (pred == batch.y).sum().item()
        total += batch.y.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

torch.save(model.state_dict(), "gnn_model.pth")
print("Model saved as gnn_model.pth")

# Export the trained model to ONNX
sample_graph = graphs[0].to(device)
dummy_input = (sample_graph.x, sample_graph.edge_index, torch.zeros(sample_graph.x.shape[0], dtype=torch.long).to(device))

torch.onnx.export(
    model, dummy_input, "gnn_model.onnx",
    export_params=True, opset_version=11, do_constant_folding=True,
    input_names=["node_features", "edge_index", "batch"],
    output_names=["output"]
)

print("Trained model saved in ONNX format as gnn_model.onnx")


Processed 159 graphs for GNN training.
Training the model...
Epoch 1/500, Loss: 0.6660
Epoch 2/500, Loss: 0.6728
Epoch 3/500, Loss: 0.6626
Epoch 4/500, Loss: 0.6617
Epoch 5/500, Loss: 0.6554
Epoch 6/500, Loss: 0.6456
Epoch 7/500, Loss: 0.6392
Epoch 8/500, Loss: 0.6392
Epoch 9/500, Loss: 0.6275
Epoch 10/500, Loss: 0.6266
Epoch 11/500, Loss: 0.6162
Epoch 12/500, Loss: 0.6177
Epoch 13/500, Loss: 0.5978
Epoch 14/500, Loss: 0.6106
Epoch 15/500, Loss: 0.6049
Epoch 16/500, Loss: 0.6144
Epoch 17/500, Loss: 0.5983
Epoch 18/500, Loss: 0.6076
Epoch 19/500, Loss: 0.5885
Epoch 20/500, Loss: 0.6044
Epoch 21/500, Loss: 0.6105
Epoch 22/500, Loss: 0.5971
Epoch 23/500, Loss: 0.5780
Epoch 24/500, Loss: 0.5983
Epoch 25/500, Loss: 0.6184
Epoch 26/500, Loss: 0.5845
Epoch 27/500, Loss: 0.5961
Epoch 28/500, Loss: 0.5772
Epoch 29/500, Loss: 0.5794
Epoch 30/500, Loss: 0.5831
Epoch 31/500, Loss: 0.5814
Epoch 32/500, Loss: 0.5721
Epoch 33/500, Loss: 0.5706
Epoch 34/500, Loss: 0.5663
Epoch 35/500, Loss: 0.5594
Epo

/usr/local/lib/python3.11/dist-packages/torch/onnx/utils.py:616: UserWarning: ONNX Preprocess - Removing mutation from node aten::scatter_add_ on block input: 'batch'. This changes graph semantics. (Triggered internally at ../torch/csrc/jit/passes/onnx/remove_inplace_ops_for_onnx.cpp:349.)
  _C._jit_pass_onnx_remove_inplace_ops_for_onnx(graph, module)
/usr/local/lib/python3.11/dist-packages/torch/onnx/symbolic_opset9.py:5333: UserWarning: Exporting aten::index operator with indices of type Byte. Only 1-D indices are supported. In any other case, this will produce an incorrect ONNX graph.
  warnings.warn(
